# Data from https://github.com/CSSEGISandData/COVID-19

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from config import db_pass


In [2]:
#data
confirmed = "Resources/time_series_19-covid-Confirmed.csv"
deaths = "Resources/time_series_19-covid-Deaths.csv"
recovered = "Resources/time_series_19-covid-Recovered.csv"


In [3]:
# create data frames
confirmed_df = pd.read_csv(confirmed)
deaths_df = pd.read_csv(deaths)
recovered_df = pd.read_csv(recovered)
confirmed_df.head(10)
# tot_locations = len(confirmed_df)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20
0,Anhui,Mainland China,31.8257,117.2264,1,9,15,39,60,70,...,989,989,989,990,990,990,990,990,990,990
1,Beijing,Mainland China,40.1824,116.4142,14,22,36,41,68,80,...,400,400,410,410,411,413,414,414,418,418
2,Chongqing,Mainland China,30.0572,107.8740,6,9,27,57,75,110,...,576,576,576,576,576,576,576,576,576,576
3,Fujian,Mainland China,26.0789,117.9874,1,5,10,18,35,59,...,294,294,296,296,296,296,296,296,296,296
4,Gansu,Mainland China,36.0611,103.8343,0,2,2,4,7,14,...,91,91,91,91,91,91,91,91,91,102
5,Guangdong,Mainland China,23.3417,113.4244,26,32,53,78,111,151,...,1347,1347,1347,1348,1349,1349,1350,1350,1350,1351
6,Guangxi,Mainland China,23.8298,108.7881,2,5,23,23,36,46,...,252,252,252,252,252,252,252,252,252,252
7,Guizhou,Mainland China,26.8154,106.8748,1,3,3,4,5,7,...,146,146,146,146,146,146,146,146,146,146
8,Hainan,Mainland China,19.1959,109.7453,4,5,8,19,22,33,...,168,168,168,168,168,168,168,168,168,168
9,Hebei,Mainland China,38.0428,114.5149,1,1,2,8,13,18,...,311,312,317,318,318,318,318,318,318,318


In [4]:
# Reset Index to have ID column
confirmed_df.reset_index(inplace=True)
confirmed_df.head()

,index,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20
0,0,Anhui,Mainland China,31.8257,117.2264,1,9,15,39,60,...,989,989,989,990,990,990,990,990,990,990
1,1,Beijing,Mainland China,40.1824,116.4142,14,22,36,41,68,...,400,400,410,410,411,413,414,414,418,418
2,2,Chongqing,Mainland China,30.0572,107.8740,6,9,27,57,75,...,576,576,576,576,576,576,576,576,576,576
3,3,Fujian,Mainland China,26.0789,117.9874,1,5,10,18,35,...,294,294,296,296,296,296,296,296,296,296
4,4,Gansu,Mainland China,36.0611,103.8343,0,2,2,4,7,...,91,91,91,91,91,91,91,91,91,102


In [5]:
# Rename to ID column
confirmed_df = confirmed_df.rename(columns={"index":"id"})
confirmed_df.head()

,id,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20
0,0,Anhui,Mainland China,31.8257,117.2264,1,9,15,39,60,...,989,989,989,990,990,990,990,990,990,990
1,1,Beijing,Mainland China,40.1824,116.4142,14,22,36,41,68,...,400,400,410,410,411,413,414,414,418,418
2,2,Chongqing,Mainland China,30.0572,107.8740,6,9,27,57,75,...,576,576,576,576,576,576,576,576,576,576
3,3,Fujian,Mainland China,26.0789,117.9874,1,5,10,18,35,...,294,294,296,296,296,296,296,296,296,296
4,4,Gansu,Mainland China,36.0611,103.8343,0,2,2,4,7,...,91,91,91,91,91,91,91,91,91,102


In [6]:
#Transpose Columns to get Dates as Row Values
df_melt = confirmed_df.melt(id_vars=['id','Province/State','Country/Region','Lat','Long'])
df_melt.columns = ['id','Province/State','Country/Region','Lat','Long','Date','confirmed_to_date']
df_melt['Date'] = pd.to_datetime(df_melt['Date'])
df_melt = df_melt.sort_values(['Date','id'])
df_melt['Count_Prev_Day'] = df_melt.sort_values('Date').groupby(['id','Province/State','Country/Region',
                                                                 'Lat','Long'])['confirmed_to_date'].shift(1)
df_melt[['Count_Prev_Day']] = df_melt[['Count_Prev_Day']].fillna(value=0)
df_melt['new_confirmed'] = df_melt['confirmed_to_date'] - df_melt['Count_Prev_Day']

df_melt.head()

,id,Province/State,Country/Region,Lat,Long,Date,confirmed_to_date,Count_Prev_Day,new_confirmed
0,0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,1,0.0,1.0
1,1,Beijing,Mainland China,40.1824,116.4142,2020-01-22,14,0.0,14.0
2,2,Chongqing,Mainland China,30.0572,107.8740,2020-01-22,6,0.0,6.0
3,3,Fujian,Mainland China,26.0789,117.9874,2020-01-22,1,0.0,1.0
4,4,Gansu,Mainland China,36.0611,103.8343,2020-01-22,0,0.0,0.0


In [7]:
#Check one location to make sure daily count working correctly
df_melt[df_melt['Province/State'] == 'Beijing'].sort_values('Date')

,id,Province/State,Country/Region,Lat,Long,Date,confirmed_to_date,Count_Prev_Day,new_confirmed
1,1,Beijing,Mainland China,40.1824,116.4142,2020-01-22,14,0.0,14.0
174,1,Beijing,Mainland China,40.1824,116.4142,2020-01-23,22,14.0,8.0
347,1,Beijing,Mainland China,40.1824,116.4142,2020-01-24,36,22.0,14.0
520,1,Beijing,Mainland China,40.1824,116.4142,2020-01-25,41,36.0,5.0
693,1,Beijing,Mainland China,40.1824,116.4142,2020-01-26,68,41.0,27.0
866,1,Beijing,Mainland China,40.1824,116.4142,2020-01-27,80,68.0,12.0
1039,1,Beijing,Mainland China,40.1824,116.4142,2020-01-28,91,80.0,11.0
1212,1,Beijing,Mainland China,40.1824,116.4142,2020-01-29,111,91.0,20.0
1385,1,Beijing,Mainland China,40.1824,116.4142,2020-01-30,114,111.0,3.0
1558,1,Beijing,Mainland China,40.1824,116.4142,2020-01-31,139,114.0,25.0


In [8]:
#Drop Count Prev Day column
confirmed_df = df_melt.drop(columns=['Count_Prev_Day'])
confirmed_df.head()

,id,Province/State,Country/Region,Lat,Long,Date,confirmed_to_date,new_confirmed
0,0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,1,1.0
1,1,Beijing,Mainland China,40.1824,116.4142,2020-01-22,14,14.0
2,2,Chongqing,Mainland China,30.0572,107.8740,2020-01-22,6,6.0
3,3,Fujian,Mainland China,26.0789,117.9874,2020-01-22,1,1.0
4,4,Gansu,Mainland China,36.0611,103.8343,2020-01-22,0,0.0


In [9]:
#Reset index to have ID column
recovered_df.reset_index(inplace=True)
recovered_df.head()

,index,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20
0,0,Anhui,Mainland China,31.8257,117.2264,0,0,0,0,0,...,712,744,792,821,868,873,917,936,956,970
1,1,Beijing,Mainland China,40.1824,116.4142,0,0,1,2,2,...,215,235,248,257,271,276,282,288,297,297
2,2,Chongqing,Mainland China,30.0572,107.8740,0,0,0,0,0,...,372,384,401,422,438,450,469,490,502,512
3,3,Fujian,Mainland China,26.0789,117.9874,0,0,0,0,0,...,199,218,228,235,243,247,255,260,270,277
4,4,Gansu,Mainland China,36.0611,103.8343,0,0,0,0,0,...,80,81,81,82,82,84,85,86,87,87


In [10]:
#Rename to have id column
recovered_df = recovered_df.rename(columns={"index":"id"})
recovered_df.head()

,id,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20
0,0,Anhui,Mainland China,31.8257,117.2264,0,0,0,0,0,...,712,744,792,821,868,873,917,936,956,970
1,1,Beijing,Mainland China,40.1824,116.4142,0,0,1,2,2,...,215,235,248,257,271,276,282,288,297,297
2,2,Chongqing,Mainland China,30.0572,107.8740,0,0,0,0,0,...,372,384,401,422,438,450,469,490,502,512
3,3,Fujian,Mainland China,26.0789,117.9874,0,0,0,0,0,...,199,218,228,235,243,247,255,260,270,277
4,4,Gansu,Mainland China,36.0611,103.8343,0,0,0,0,0,...,80,81,81,82,82,84,85,86,87,87


In [11]:
#transpose date columns to row values
df_melt = recovered_df.melt(id_vars=['id','Province/State','Country/Region','Lat','Long'])
df_melt.columns = ['id','Province/State','Country/Region','Lat','Long','Date','recovered_to_date']
df_melt['Date'] = pd.to_datetime(df_melt['Date'])
df_melt = df_melt.sort_values(['Date','id'])
df_melt['Count_Prev_Day'] = df_melt.sort_values('Date').groupby(['id','Province/State','Country/Region',
                                                                 'Lat','Long'])['recovered_to_date'].shift(1)
df_melt[['Count_Prev_Day']] = df_melt[['Count_Prev_Day']].fillna(value=0)
df_melt['new_recovered'] = df_melt['recovered_to_date'] - df_melt['Count_Prev_Day']

df_melt.head()

,id,Province/State,Country/Region,Lat,Long,Date,recovered_to_date,Count_Prev_Day,new_recovered
0,0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,0,0.0,0.0
1,1,Beijing,Mainland China,40.1824,116.4142,2020-01-22,0,0.0,0.0
2,2,Chongqing,Mainland China,30.0572,107.8740,2020-01-22,0,0.0,0.0
3,3,Fujian,Mainland China,26.0789,117.9874,2020-01-22,0,0.0,0.0
4,4,Gansu,Mainland China,36.0611,103.8343,2020-01-22,0,0.0,0.0


In [12]:
#Filter on one location to make sure working correctly
df_melt[df_melt['Province/State'] == 'Beijing'].sort_values('Date')

,id,Province/State,Country/Region,Lat,Long,Date,recovered_to_date,Count_Prev_Day,new_recovered
1,1,Beijing,Mainland China,40.1824,116.4142,2020-01-22,0,0.0,0.0
174,1,Beijing,Mainland China,40.1824,116.4142,2020-01-23,0,0.0,0.0
347,1,Beijing,Mainland China,40.1824,116.4142,2020-01-24,1,0.0,1.0
520,1,Beijing,Mainland China,40.1824,116.4142,2020-01-25,2,1.0,1.0
693,1,Beijing,Mainland China,40.1824,116.4142,2020-01-26,2,2.0,0.0
866,1,Beijing,Mainland China,40.1824,116.4142,2020-01-27,2,2.0,0.0
1039,1,Beijing,Mainland China,40.1824,116.4142,2020-01-28,4,2.0,2.0
1212,1,Beijing,Mainland China,40.1824,116.4142,2020-01-29,4,4.0,0.0
1385,1,Beijing,Mainland China,40.1824,116.4142,2020-01-30,4,4.0,0.0
1558,1,Beijing,Mainland China,40.1824,116.4142,2020-01-31,5,4.0,1.0


In [13]:
#Drop Prev Day Cout column
recovered_df = df_melt.drop(columns=['Count_Prev_Day'])
recovered_df.head()

,id,Province/State,Country/Region,Lat,Long,Date,recovered_to_date,new_recovered
0,0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,0,0.0
1,1,Beijing,Mainland China,40.1824,116.4142,2020-01-22,0,0.0
2,2,Chongqing,Mainland China,30.0572,107.8740,2020-01-22,0,0.0
3,3,Fujian,Mainland China,26.0789,117.9874,2020-01-22,0,0.0
4,4,Gansu,Mainland China,36.0611,103.8343,2020-01-22,0,0.0


In [14]:
#Reset index and save prvious index as id column
deaths_df.reset_index(inplace=True)
deaths_df.head()

,index,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20
0,0,Anhui,Mainland China,31.8257,117.2264,0,0,0,0,0,...,6,6,6,6,6,6,6,6,6,6
1,1,Beijing,Mainland China,40.1824,116.4142,0,0,0,0,0,...,4,4,5,7,8,8,8,8,8,8
2,2,Chongqing,Mainland China,30.0572,107.8740,0,0,0,0,0,...,6,6,6,6,6,6,6,6,6,6
3,3,Fujian,Mainland China,26.0789,117.9874,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
4,4,Gansu,Mainland China,36.0611,103.8343,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2


In [15]:
#Rename to have id column
deaths_df = deaths_df.rename(columns={"index":"id"})
deaths_df.head()

,id,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20
0,0,Anhui,Mainland China,31.8257,117.2264,0,0,0,0,0,...,6,6,6,6,6,6,6,6,6,6
1,1,Beijing,Mainland China,40.1824,116.4142,0,0,0,0,0,...,4,4,5,7,8,8,8,8,8,8
2,2,Chongqing,Mainland China,30.0572,107.8740,0,0,0,0,0,...,6,6,6,6,6,6,6,6,6,6
3,3,Fujian,Mainland China,26.0789,117.9874,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
4,4,Gansu,Mainland China,36.0611,103.8343,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2


In [16]:
#Transpose date columns to have as row values
df_melt = deaths_df.melt(id_vars=['id','Province/State','Country/Region','Lat','Long'])
df_melt.columns = ['id','Province/State','Country/Region','Lat','Long','Date','deaths_to_date']
df_melt['Date'] = pd.to_datetime(df_melt['Date'])
df_melt = df_melt.sort_values(['Date','id'])
df_melt['Count_Prev_Day'] = df_melt.sort_values('Date').groupby(['id','Province/State','Country/Region',
                                                                 'Lat','Long'])['deaths_to_date'].shift(1)
df_melt[['Count_Prev_Day']] = df_melt[['Count_Prev_Day']].fillna(value=0)
df_melt['new_deaths'] = df_melt['deaths_to_date'] - df_melt['Count_Prev_Day']

df_melt.head()

,id,Province/State,Country/Region,Lat,Long,Date,deaths_to_date,Count_Prev_Day,new_deaths
0,0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,0,0.0,0.0
1,1,Beijing,Mainland China,40.1824,116.4142,2020-01-22,0,0.0,0.0
2,2,Chongqing,Mainland China,30.0572,107.8740,2020-01-22,0,0.0,0.0
3,3,Fujian,Mainland China,26.0789,117.9874,2020-01-22,0,0.0,0.0
4,4,Gansu,Mainland China,36.0611,103.8343,2020-01-22,0,0.0,0.0


In [17]:
#Filter on one location to make sure working correctly
df_melt[df_melt['Province/State'] == 'Beijing'].sort_values('Date')

,id,Province/State,Country/Region,Lat,Long,Date,deaths_to_date,Count_Prev_Day,new_deaths
1,1,Beijing,Mainland China,40.1824,116.4142,2020-01-22,0,0.0,0.0
174,1,Beijing,Mainland China,40.1824,116.4142,2020-01-23,0,0.0,0.0
347,1,Beijing,Mainland China,40.1824,116.4142,2020-01-24,0,0.0,0.0
520,1,Beijing,Mainland China,40.1824,116.4142,2020-01-25,0,0.0,0.0
693,1,Beijing,Mainland China,40.1824,116.4142,2020-01-26,0,0.0,0.0
866,1,Beijing,Mainland China,40.1824,116.4142,2020-01-27,1,0.0,1.0
1039,1,Beijing,Mainland China,40.1824,116.4142,2020-01-28,1,1.0,0.0
1212,1,Beijing,Mainland China,40.1824,116.4142,2020-01-29,1,1.0,0.0
1385,1,Beijing,Mainland China,40.1824,116.4142,2020-01-30,1,1.0,0.0
1558,1,Beijing,Mainland China,40.1824,116.4142,2020-01-31,1,1.0,0.0


In [18]:
#Drop prev day count column
deaths_df = df_melt.drop(columns=['Count_Prev_Day'])
deaths_df.head()

,id,Province/State,Country/Region,Lat,Long,Date,deaths_to_date,new_deaths
0,0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,0,0.0
1,1,Beijing,Mainland China,40.1824,116.4142,2020-01-22,0,0.0
2,2,Chongqing,Mainland China,30.0572,107.8740,2020-01-22,0,0.0
3,3,Fujian,Mainland China,26.0789,117.9874,2020-01-22,0,0.0
4,4,Gansu,Mainland China,36.0611,103.8343,2020-01-22,0,0.0


In [19]:
# Merge confirmed and deaths df on all common columns
semicombined_df = pd.merge(confirmed_df, deaths_df, on=['id','Province/State','Country/Region','Lat','Long', 'Date'])
semicombined_df.head()

,id,Province/State,Country/Region,Lat,Long,Date,confirmed_to_date,new_confirmed,deaths_to_date,new_deaths
0,0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,1,1.0,0,0.0
1,1,Beijing,Mainland China,40.1824,116.4142,2020-01-22,14,14.0,0,0.0
2,2,Chongqing,Mainland China,30.0572,107.8740,2020-01-22,6,6.0,0,0.0
3,3,Fujian,Mainland China,26.0789,117.9874,2020-01-22,1,1.0,0,0.0
4,4,Gansu,Mainland China,36.0611,103.8343,2020-01-22,0,0.0,0,0.0


In [20]:
#Merge semimerged df w recovered df on all common columns
combined_df = pd.merge(semicombined_df, recovered_df, on=['id','Province/State','Country/Region','Lat','Long','Date'])
combined_df.head()

,id,Province/State,Country/Region,Lat,Long,Date,confirmed_to_date,new_confirmed,deaths_to_date,new_deaths,recovered_to_date,new_recovered
0,0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,1,1.0,0,0.0,0,0.0
1,1,Beijing,Mainland China,40.1824,116.4142,2020-01-22,14,14.0,0,0.0,0,0.0
2,2,Chongqing,Mainland China,30.0572,107.8740,2020-01-22,6,6.0,0,0.0,0,0.0
3,3,Fujian,Mainland China,26.0789,117.9874,2020-01-22,1,1.0,0,0.0,0,0.0
4,4,Gansu,Mainland China,36.0611,103.8343,2020-01-22,0,0.0,0,0.0,0,0.0


In [43]:
#Rename columns to format for SQL
final_df = combined_df.rename(columns={"id":"id_loc","Province/State": "province_state", "Country/Region": "country_region", "Lat":"lat", "Long": "long", "Date": "date"})
final_df.head()

,id_loc,province_state,country_region,lat,long,date,confirmed_to_date,new_confirmed,deaths_to_date,new_deaths,recovered_to_date,new_recovered
0,0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,1,1.0,0,0.0,0,0.0
1,1,Beijing,Mainland China,40.1824,116.4142,2020-01-22,14,14.0,0,0.0,0,0.0
2,2,Chongqing,Mainland China,30.0572,107.8740,2020-01-22,6,6.0,0,0.0,0,0.0
3,3,Fujian,Mainland China,26.0789,117.9874,2020-01-22,1,1.0,0,0.0,0,0.0
4,4,Gansu,Mainland China,36.0611,103.8343,2020-01-22,0,0.0,0,0.0,0,0.0


In [44]:
# conection to SPQ database 
rds_connection_string =f"postgres:{db_pass}@localhost:5432/COVID19_DB"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [45]:
engine.table_names()


['covid19']

In [46]:
# table with all data

final_df.to_sql(name='covid19', con=engine, if_exists='append', index=False)

In [48]:
## Confirm data has been queried properly
pd.read_sql_query('select * from covid19', con=engine).head()

,id,id_loc,province_state,country_region,lat,long,date,confirmed_to_date,new_confirmed,deaths_to_date,new_deaths,recovered_to_date,new_recovered
0,1,0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,1,1,0,0,0,0
1,2,1,Beijing,Mainland China,40.1824,116.4142,2020-01-22,14,14,0,0,0,0
2,3,2,Chongqing,Mainland China,30.0572,107.8740,2020-01-22,6,6,0,0,0,0
3,4,3,Fujian,Mainland China,26.0789,117.9874,2020-01-22,1,1,0,0,0,0
4,5,4,Gansu,Mainland China,36.0611,103.8343,2020-01-22,0,0,0,0,0,0
